# CrewAI API Reference

This notebook demonstrates the native CrewAI API and how to use it for building multi-agent systems.

## Overview

CrewAI provides a framework for creating AI agents that can collaborate on tasks. The core components are:
- **Agent**: An AI worker with a role, goal, and tools
- **Task**: A work item assigned to an agent
- **Crew**: Orchestrates agents and tasks
- **LLM**: Language model configuration
- **Tool**: Functions agents can use


In [ ]:
# Import CrewAI components
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool

import os
print("CrewAI imported successfully")


## 1. LLM Configuration

First, configure the Language Model that agents will use.


In [ ]:
# Set your OpenAI API key
os.environ['OPENAI_API_KEY'] = 'your-api-key-here'  # Replace with your actual key

# Create LLM instance
llm = LLM(
    model="gpt-4o",  # or "gpt-4", "gpt-3.5-turbo"
    api_key=os.getenv("OPENAI_API_KEY")
)

print("LLM configured")


## 2. Creating Tools

Tools are functions that agents can use to interact with data or external systems.


In [ ]:
@tool("get_weather")
def get_weather(city: str) -> str:
    """
    Get the current weather for a city.
    
    Args:
        city: Name of the city
    
    Returns:
        Weather information as a string
    """
    # Simulated weather data
    return f"The weather in {city} is sunny, 72°F"

@tool("calculate")
def calculate(expression: str) -> str:
    """
    Calculate a mathematical expression.
    
    Args:
        expression: Mathematical expression to evaluate (e.g., "2 + 2")
    
    Returns:
        Result of the calculation
    """
    try:
        result = eval(expression)  # In production, use a safer evaluator
        return f"Result: {result}"
    except:
        return f"Error: Could not evaluate '{expression}'"

# List of tools for agents
tools = [get_weather, calculate]

print("Tools created:", [t.name for t in tools])


## 3. Creating Agents

Agents are AI workers with specific roles and capabilities.


In [ ]:
# Create a research agent
researcher = Agent(
    role="Research Analyst",
    goal="Gather and analyze information",
    backstory="You are an expert researcher with years of experience in data analysis.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tools
)

# Create a writer agent
writer = Agent(
    role="Content Writer",
    goal="Create engaging written content",
    backstory="You are a skilled writer who transforms information into compelling narratives.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[]  # Writer doesn't need tools
)

print("Agents created:")
print(f"  - {researcher.role}")
print(f"  - {writer.role}")


## 4. Creating Tasks

Tasks define what agents should do.


In [ ]:
# Task 1: Research task (independent)
research_task = Task(
    description="""
    Research the weather in New York and calculate 15 * 23.
    Use the available tools to get this information.
    """,
    agent=researcher,
    expected_output="Weather information and calculation result"
)

# Task 2: Writing task (depends on research_task)
writing_task = Task(
    description="""
    Write a short paragraph about the weather in New York and the calculation result.
    Use the research results from the previous task.
    """,
    agent=writer,
    context=[research_task],  # This task depends on research_task
    expected_output="A well-written paragraph combining weather and calculation information"
)

print("Tasks created:")
print(f"  - Research Task (independent)")
print(f"  - Writing Task (depends on Research)")


## 5. Creating and Executing a Crew

A Crew orchestrates agents and tasks into a workflow.


In [ ]:
# Create crew
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    process=Process.sequential,  # Sequential execution (respects dependencies)
    verbose=True
)

print("Crew created")
print("\nExecuting crew...")
print("-" * 70)

# Execute the crew
result = crew.kickoff()

print("-" * 70)
print("\nRESULTS:")
print("=" * 70)
print(result)


## 6. Accessing Individual Task Outputs

You can access outputs from individual tasks.


In [ ]:
# Access individual task outputs
if hasattr(result, 'tasks_output') and result.tasks_output:
    print("Task 1 (Research) Output:")
    print(result.tasks_output[0])
    print("\n" + "-" * 70)
    print("\nTask 2 (Writing) Output:")
    print(result.tasks_output[1])
else:
    print("Full result:", str(result))


## 7. Parallel Task Execution

Tasks without dependencies run in parallel.


In [ ]:
# Create two independent tasks (no context dependencies)
task1 = Task(
    description="Get weather for London",
    agent=researcher,
    expected_output="Weather information for London"
)

task2 = Task(
    description="Get weather for Tokyo",
    agent=researcher,
    expected_output="Weather information for Tokyo"
)

# Both tasks run in parallel since neither depends on the other
parallel_crew = Crew(
    agents=[researcher],
    tasks=[task1, task2],
    process=Process.sequential,  # Sequential process still handles parallel execution
    verbose=True
)

print("Executing parallel tasks...")
print("-" * 70)

result = parallel_crew.kickoff()

print("-" * 70)
print("\nPARALLEL RESULTS:")
print("=" * 70)
print(result)


## Summary

This notebook demonstrated:
- LLM configuration
- Tool creation with `@tool` decorator
- Agent creation with roles, goals, and backstories
- Task creation with dependencies
- Crew orchestration and execution
- Accessing task outputs
- Parallel vs sequential execution

For more details, see `crewai.API.md` for complete API documentation.
